## Notebook for feature engineering for gym-motion-ai

Below cell will encode video to json [X, Y, Z, Visibility, Landmark]

In [1]:
import cv2
import mediapipe as mp
import json

In [2]:

def save_to_json(json_file_path, data):
    with open(json_file_path, 'a') as json_file:
        json.dump(data, json_file)
        json_file.write('\n')

# Function to process frames
def process_frame(frame, pose):
    # Convert the BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame using MediaPipe Pose
    results = pose.process(rgb_frame)

    # Do something with the results (e.g., draw pose landmarks)
    if results.pose_landmarks:
        # Draw landmarks on the frame
        keypoints = []
        index = 0
        for data_point in results.pose_landmarks.landmark:
            keypoints.append({
                                 'X': data_point.x,
                                 'Y': data_point.y,
                                 'Z': data_point.z,
                                 'Visibility': data_point.visibility,
                                 'Landmark':index
                                 })
            index +=1
        
        # Save coordinates to JSON file
        save_to_json('barbell_dead_row_65906101_s05_13ang.json', keypoints)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    return frame

# Initialize MediaPipe Pose and Drawing modules
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Video file path
video_path = "videos/barbell_dead_row_65906101_s05.mp4"
output_path = "json/ann_" + video_path

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
else: 
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Create VideoWriter object to save the processed video
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    # Initialize MediaPipe Pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()

        # Break the loop if the video is finished
        if not ret:
            break

        # Process the frame
        processed_frame = process_frame(frame, pose)

        # Display the processed frame (optional)
        cv2.imshow("Processed Frame", processed_frame)

        # Write the processed frame to the output video file
        out.write(processed_frame)

        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the VideoCapture, VideoWriter, and Pose objects
    cap.release()
    out.release()
    pose.close()

    # Destroy any OpenCV windows
    cv2.destroyAllWindows()

____

# Process JSON

In [6]:
import json
import matplotlib.pyplot as plt
import pandas as pd

# Read the JSON file
file_path = 'json/barbell_dead_row_65906101_s05_13ang.json'

with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

In [7]:
data[0]

[{'X': 0.5437556505203247,
  'Y': 0.2606537938117981,
  'Z': -0.2704053819179535,
  'Visibility': 0.999983549118042,
  'Landmark': 0},
 {'X': 0.5516217947006226,
  'Y': 0.25203752517700195,
  'Z': -0.2619931399822235,
  'Visibility': 0.9999545812606812,
  'Landmark': 1},
 {'X': 0.5554302930831909,
  'Y': 0.25224271416664124,
  'Z': -0.2620546221733093,
  'Visibility': 0.9999605417251587,
  'Landmark': 2},
 {'X': 0.5596731305122375,
  'Y': 0.25260642170906067,
  'Z': -0.26209303736686707,
  'Visibility': 0.9999667406082153,
  'Landmark': 3},
 {'X': 0.5429314374923706,
  'Y': 0.2517625689506531,
  'Z': -0.25376853346824646,
  'Visibility': 0.9999258518218994,
  'Landmark': 4},
 {'X': 0.5408254861831665,
  'Y': 0.251723051071167,
  'Z': -0.25376996397972107,
  'Visibility': 0.9999094009399414,
  'Landmark': 5},
 {'X': 0.5391855835914612,
  'Y': 0.2517637610435486,
  'Z': -0.2537843883037567,
  'Visibility': 0.9998918771743774,
  'Landmark': 6},
 {'X': 0.5680534839630127,
  'Y': 0.25814896

In [8]:
columns = ['Data','Frame', 'left_arm','right_arm','left_elbow','right_elbow','left_waist_leg','right_waist_leg','left_knee','right_kneee','leftup_chest_inside','rightup_chest_inside','leftlow_chest_inside','rightlow_chest_inside','leg_spread']
df = pd.DataFrame(columns=columns)
df

,Data,Frame,left_arm,right_arm,left_elbow,right_elbow,left_waist_leg,right_waist_leg,left_knee,right_kneee,leftup_chest_inside,rightup_chest_inside,leftlow_chest_inside,rightlow_chest_inside,leg_spread


In [9]:
# iterate over the dataframe 'Data' and calculate angles
import numpy as np

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# joints - 26, 24, 23, 25
def calculate_leg_spread(a, b, c, d):
    # Find the midpoint between b and c
    mid_point_bc = [(b[0] + c[0]) / 2, (b[1] + c[1]) / 2]

    # Calculate the angle between a, (b, c), and d using the midpoint as the vertex
    angle = calculate_angle(a, mid_point_bc, d)

    return angle

def getLandmarkXYZ(coordinates, landmark):
    for coord in coordinates:
        if coord['Landmark'] == landmark:
            return [coord['X'], coord['Y']]

def calculateAngles(coords, joints):
    angles = []
    for joint in joints:
        if len(joint) == 3:
            landmark_first = getLandmarkXYZ(coords, joint[0])
            landmark_mid = getLandmarkXYZ(coords, joint[1])
            landmark_end = getLandmarkXYZ(coords, joint[2])

            angle = calculate_angle(landmark_first, landmark_mid, landmark_end)
        else:
            landmark_l_knee = getLandmarkXYZ(coords, joint[0])
            landmark_l_hip = getLandmarkXYZ(coords, joint[1])
            landmark_r_hip = getLandmarkXYZ(coords, joint[2])
            landmark_r_knee = getLandmarkXYZ(coords, joint[3])
            
            angle = calculate_leg_spread(landmark_l_knee, landmark_l_hip, landmark_r_hip, landmark_r_knee)
        angles.append(angle)
        
    return angles
        
all_angles = [(14,12,24), (13,11,23), (16,14,12),(15,13,11),(12,24,26), (11,23,25), (24,26,28), (23, 25,27),(11,12,24), (12, 11, 23), (26, 24, 23), (25, 23, 24), (26, 24, 23, 25)]
angle_columns_names = ['left_arm','right_arm','left_elbow','right_elbow','left_waist_leg','right_waist_leg','left_knee','right_knee','leftup_chest_inside','rightup_chest_inside','leftlow_chest_inside','rightlow_chest_inside']      
len(all_angles), len(angle_columns_names)

(13, 12)

In [10]:
# Map json data to dataframe

frame = 0
for row in data:
    angles = calculateAngles(row, all_angles)
    # angles will have 12 values
    
    df.loc[len(df)] = [row, frame, *angles]
    frame +=1

In [11]:
df

,Data,Frame,left_arm,right_arm,left_elbow,right_elbow,left_waist_leg,right_waist_leg,left_knee,right_kneee,leftup_chest_inside,rightup_chest_inside,leftlow_chest_inside,rightlow_chest_inside,leg_spread
0,"[{'X': 0.5437556505203247, 'Y': 0.260653793811...",0,16.967365,20.138432,169.483217,174.431905,171.884929,164.911296,178.278814,179.377459,84.921846,80.475747,89.705913,98.895454,33.049457
1,"[{'X': 0.5437383651733398, 'Y': 0.260431408882...",1,16.936865,20.042459,169.550945,174.445865,171.958123,164.523618,178.048408,178.356059,84.965892,80.538991,89.807621,99.215520,33.767571
2,"[{'X': 0.543701171875, 'Y': 0.2603459060192108...",2,16.893990,20.000677,169.536093,174.428564,172.033954,164.174630,177.901790,177.625834,85.017512,80.553714,89.832134,99.530506,34.271913
3,"[{'X': 0.5436867475509644, 'Y': 0.260182350873...",3,16.878244,19.917658,169.554811,174.474301,172.072884,164.028646,177.794069,177.370415,85.056615,80.575980,89.909441,99.621625,34.593849
4,"[{'X': 0.5436744689941406, 'Y': 0.260061174631...",4,16.868878,19.872644,169.541542,174.476371,172.091447,163.984656,177.616330,177.251948,85.067146,80.601857,89.924642,99.668258,34.747010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
896,"[{'X': 0.5384734272956848, 'Y': 0.260983347892...",896,15.042698,19.051447,168.117955,176.804550,171.422058,166.843269,177.337546,179.791640,82.756601,83.244517,90.579875,97.155915,33.315715
897,"[{'X': 0.538462221622467, 'Y': 0.2609836459159...",897,14.961738,19.032719,168.005807,176.753604,171.331839,166.920824,177.344001,179.734758,82.740337,83.287227,90.644347,97.130554,33.334234
898,"[{'X': 0.5384187698364258, 'Y': 0.260977119207...",898,14.898283,19.013885,167.889568,176.707580,171.222560,167.018620,177.310482,179.731383,82.705422,83.334710,90.728935,97.070016,33.309565
899,"[{'X': 0.5383760333061218, 'Y': 0.260975331068...",899,14.791526,19.026225,167.771745,176.614318,171.167019,167.099506,177.286525,179.716593,82.724440,83.359966,90.790810,97.027071,33.340918


# Angles

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [12]:
df.to_csv('json/barbell_dead_row_65906101_s05_13ang.csv', index=False)

In [35]:
# joints we need to process

upper = [(14,12,24), (13,11,23), (16,14,12),(15,13,11)]
lower = [(12,24,26), (11,23,25), (24,26,28), (23, 25,27)]

upper_inside = [(11,12,24), (12, 11, 23)]
lower_inside = [(26, 24, 23), (25, 23, 24)]

# Post process

In [13]:
leftArmFps = df[['right_kneee','Frame']]
leftArmFps

,right_kneee,Frame
0,179.377459,0
1,178.356059,1
2,177.625834,2
3,177.370415,3
4,177.251948,4
...,...,...
896,179.791640,896
897,179.734758,897
898,179.731383,898
899,179.716593,899


In [14]:
import cv2
import numpy as np
import pandas as pd

# Create a blank white image
image = np.ones((500, 500, 3), dtype=np.uint8) * 255

# Define the coordinates for the horizontal line
line_start_horizontal = (100, 250)
line_end_horizontal = (200, 250)

# Initialize the angle and frame index
angle = np.radians(30)
frame_index = 0

# Function to update the angle based on the DataFrame
def update_angle(frame):
    global angle, frame_index
    if frame_index < len(df):
        angle = np.radians(df['right_kneee'].iloc[frame_index])
        frame_index += 1

# Main loop to continuously update and draw the lines
while True:
    # Fill the image with a white background
    image = np.ones((500, 500, 3), dtype=np.uint8) * 255

    # Draw the horizontal line in red
    cv2.line(image, line_start_horizontal, line_end_horizontal, (0, 0, 255), 2)

    # Calculate the end coordinates for the second line
    line_length = 150  # Adjust the length as needed
    line_end_x = int(line_end_horizontal[0] + line_length * np.cos(angle))
    line_end_y = int(line_end_horizontal[1] - line_length * np.sin(angle))
    line_end_2 = (line_end_x, line_end_y)

    # Draw the second line in blue
    cv2.line(image, line_end_horizontal, line_end_2, (255, 0, 0), 2)

    # Display the angle in the top left corner
    angle_text = f'Angle: {np.degrees(angle):.2f} degrees'
    cv2.putText(image, angle_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Update the angle based on the DataFrame
    update_angle(frame_index)

    # Show the image and wait for 20 milliseconds
    cv2.imshow('Draw Lines', image)
    key = cv2.waitKey(20)

    # Check if the 'Esc' key is pressed to exit the loop
    if key == 27:
        break

cv2.destroyAllWindows()

In [15]:
import cv2
import numpy as np
import pandas as pd

# Assuming df is your DataFrame with angle values
# You may need to replace 'right_kneee' with the actual column names

# Function to draw a two-line animation with angle and name text
def draw_animation(image, line_start, angle, angle_name):
    line_length = 50

    # Draw the first line in red
    line_end_1 = (int(line_start[0] + line_length), line_start[1])
    cv2.line(image, line_start, line_end_1, (0, 0, 255), 2)

    # Draw the second line at an angle in blue
    line_end_2 = (
        int(line_end_1[0] + line_length * np.cos(np.radians(angle))),
        int(line_end_1[1] - line_length * np.sin(np.radians(angle)))
    )
    cv2.line(image, line_end_1, line_end_2, (255, 0, 0), 2)

    # Display angle and name text
    angle_text = f'{angle_name}:{(angle):.2f}deg'
    cv2.putText(image, angle_text, (line_start[0]-40, line_start[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

# Create a blank white landscape image
landscape_image = np.ones((500, 850, 3), dtype=np.uint8) * 255

# Define the initial starting points for each animation in a 4x3 grid
start_points_grid = [(50 + (i % 4) * 200, 50 + (i // 4) * 150) for i in range(12)]

# Extract relevant columns from the DataFrame
angle_columns = [
    'left_arm', 'right_arm', 'left_elbow', 'right_elbow',
    'left_waist_leg', 'right_waist_leg', 'left_knee', 'right_kneee',
    'leftup_chest_inside', 'rightup_chest_inside', 'leftlow_chest_inside', 'rightlow_chest_inside'
]

# Initialize frame index
frame_index = 0

# Main loop to continuously update and draw the lines
while True:
    # Fill the landscape image with a white background
    landscape_image = np.ones((500, 850, 3), dtype=np.uint8) * 255

    # Draw animations for each angle in a 4x3 grid
    for i, (start_point, angle_column) in enumerate(zip(start_points_grid, angle_columns)):
        angle_value = df[angle_column].iloc[frame_index % len(df)]
        draw_animation(landscape_image, start_point, angle_value, angle_column)

    # Show the landscape image and wait for 20 milliseconds
    cv2.imshow('Grid of Animations', landscape_image)
    key = cv2.waitKey(20)

    # Update frame index for the next iteration
    frame_index += 1

    # Check if the 'Esc' key is pressed to exit the loop
    if key == 27:
        break

cv2.destroyAllWindows()


___

# More features

    Temporal Features:

        Rate of Change of Angles (Angular Velocity): Compute the derivative of joint angles with respect to frame number. This will give you the rate at which each joint angle is changing over time, providing insights into the dynamics of the motion.
        Angular Velocity=ΔJoint AngleΔFrameAngular Velocity=ΔFrameΔJoint Angle​

        Acceleration of Joint Angles: You can further calculate the second derivative of joint angles with respect to frame number, giving you information about the acceleration of the joint movements.
        Angular Acceleration=Δ2Joint AngleΔFrame2Angular Acceleration=ΔFrame2Δ2Joint Angle​

    Statistical Features:

        Mean and Standard Deviation: Compute the mean and standard deviation of joint angles. These statistics can provide a summary of the central tendency and variability of the joint angles, respectively.

        Range of Joint Angles: Calculate the range (difference between the maximum and minimum values) for each joint angle. This can give you an idea of how much variation exists in each joint movement.

    Relative Joint Positions:
        Relative Angles: If your dataset includes angles of multiple joints, you can create relative angles by calculating the difference between the angles of two joints. This may provide information about the coordination or relative positioning of different joints.

    Frame-based Features:
        Frame Number Normalization: If the frame numbers have a wide range, you might want to normalize them to a common scale (e.g., between 0 and 1) to avoid biasing the model towards larger frame numbers.

    Smoothing Techniques:
        Moving Averages or Filters: Apply smoothing techniques like moving averages or filters to reduce noise in the joint angle data. This can help in identifying more prominent trends in the motion.

In [42]:
import pandas as pd


df['Angular_Velocity'] = df['right_kneee'].diff() / df['Frame'].diff()
df['Angular_Acceleration'] = df['Angular_Velocity'].diff() / df['Frame'].diff()

# The first row of 'Angular_Velocity' and 'Angular_Acceleration' will be NaN due to the diff operation
# You may choose to fill or drop these rows based on your preference
# For example, you can fill NaN values with 0
df['Angular_Velocity'].fillna(0, inplace=True)
df['Angular_Acceleration'].fillna(0, inplace=True)


In [43]:
df

,Data,Frame,left_arm,right_arm,left_elbow,right_elbow,left_waist_leg,right_waist_leg,left_knee,right_kneee,leftup_chest_inside,rightup_chest_inside,leftlow_chest_inside,rightlow_chest_inside,Angular_Velocity,Angular_Acceleration
0,"[{'X': 0.4833790361881256, 'Y': 0.294812440872...",0,23.601473,12.751360,174.853980,169.601054,167.159386,168.788805,179.287017,179.716307,83.319455,82.597954,98.742145,91.227074,0.000000,0.000000
1,"[{'X': 0.4835801124572754, 'Y': 0.294852554798...",1,23.621931,12.894650,174.758119,169.773449,167.661377,168.605542,179.996630,179.566293,83.065925,82.788702,97.930294,91.657414,-0.150014,0.000000
2,"[{'X': 0.4838849902153015, 'Y': 0.294873952865...",2,23.694459,12.969590,174.690819,169.926532,168.012126,168.577495,179.423562,179.547380,82.921925,82.906226,97.333005,91.905525,-0.018913,0.131101
3,"[{'X': 0.4841047525405884, 'Y': 0.294852823019...",3,23.705468,12.996209,174.666872,169.986297,168.272435,168.570871,179.120662,179.576211,82.802500,83.028339,96.846097,92.141435,0.028831,0.047744
4,"[{'X': 0.4841686189174652, 'Y': 0.294777154922...",4,23.705424,13.053929,174.655856,170.103835,168.348729,168.524051,179.029729,179.484526,82.756730,83.068577,96.691334,92.261193,-0.091685,-0.120516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,"[{'X': 0.48054423928260803, 'Y': 0.29292860627...",1104,20.421291,13.460143,178.269920,169.739278,174.123672,169.636900,176.826247,178.628721,84.002710,81.731117,92.351064,89.622190,-0.026152,0.002405
1105,"[{'X': 0.480549693107605, 'Y': 0.2930911183357...",1105,20.522239,13.425830,178.330427,169.662291,174.101794,169.660426,176.844601,178.603554,83.976487,81.754617,92.383352,89.585531,-0.025167,0.000985
1106,"[{'X': 0.48055487871170044, 'Y': 0.29342129826...",1106,20.581019,13.426049,178.429792,169.644319,174.104322,169.675202,176.847499,178.586911,83.927227,81.816159,92.348183,89.615680,-0.016642,0.008525
1107,"[{'X': 0.4805610775947571, 'Y': 0.294127196073...",1107,20.665500,13.394933,178.543092,169.570020,174.103511,169.678396,176.851154,178.573929,83.877490,81.870203,92.348227,89.617560,-0.012983,0.003659


In [44]:
df.to_csv('json/angular_angles.csv', index=False)

___

In [4]:
import cv2

def list_video_captures():
    index = 0
    cap_list = []

    while True:
        # Try to open the video capture device at the current index
        cap = cv2.VideoCapture(index)

        # Check if the capture device was successfully opened
        if not cap.isOpened():
            break

        # Get information about the capture device
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Print information about the capture device
        print(f"Capture Device {index}: {width}x{height}")

        # Release the capture device
        cap.release()

        # Move to the next index
        index += 1

    print("Finished listing video captures.")

# Call the function to list all video captures
list_video_captures()


Capture Device 0: 640x480
Capture Device 1: 960x540
Capture Device 2: 640x480
Finished listing video captures.


In [3]:
# VIDEO FEED
cap = cv2.VideoCapture(2)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()